# Train first set of one vs. rest (OVR) classifiers.

We train a set of classifiers that are used for computations of $ClassSim$ that is defined in our paper to estimate a similarity between classes.  
These classifiers are also used for classification tasks.  
Modules used in this notebook are defined in files in *models* directory.

## Set up

In [1]:
import os
import sys

import numpy as np

import pandas as pd
import glob

import warnings
warnings.filterwarnings('ignore')

In [2]:
BASE_MODEL_PATH="trained_model"
%mkdir -p $BASE_MODEL_PATH

In [3]:
from models.modelutils import ModelCompiler

Using TensorFlow backend.


In [4]:
compiler = ModelCompiler(BASE_MODEL_PATH)

In [5]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()

Load category information and all image paths.

In [6]:
from models.modelutils import dir2filedict, split_fdict

In [7]:
trdict = dir2filedict("data_fgvc/train")

In [8]:
valdict = dir2filedict("data_fgvc/valid")

In [18]:
categories = [str(i) for i in range(0, 100)]

In [19]:
categories[99]

'99'

In [13]:
valdict['0'][0:5]

['data_fgvc/valid/0/1319365.jpg',
 'data_fgvc/valid/0/0062781.jpg',
 'data_fgvc/valid/0/1042021.jpg',
 'data_fgvc/valid/0/0602177.jpg',
 'data_fgvc/valid/0/0817494.jpg']

Here is expected outputs.   
All the outputs in {*train.ipynb*, *classifier_similarity.ipynb*, *train_multiclass_classifier.ipynb*, *train_second.ipynb*} must be the same. 

['data_fgvc/valid/0/1319365.jpg',  
 'data_fgvc/valid/0/0062781.jpg',  
 'data_fgvc/valid/0/1042021.jpg',  
 'data_fgvc/valid/0/0602177.jpg',  
 'data_fgvc/valid/0/0817494.jpg']

## Train classifiers

In [14]:
from models.one_vs_all import OneVsAllModelTrainer
from models.modelutils import split_files

In [15]:
trainer = OneVsAllModelTrainer(TRAIN_DATAGEN, VALID_DATAGEN)

In [16]:
def train_one_category(cat, epoch=5):
    model_path = "{}/modelfgcv_{}".format(BASE_MODEL_PATH, cat)
    model = compiler.generate_compiled_model(model_path)
    
    trainer.set_model(model)
    trainer.set_savepath(model_path)
    
    true_train, false_train = split_files(cat, trdict)
    true_valid, false_valid = split_files(cat, valdict)
    
    trainer.set_dataset_files(true_train, false_train, true_valid, false_valid)
    trainer.train_model(eachepochs=epoch)

In [ ]:
train_one_category(categories[0])

Epoch 1/5
7/6 [==============================] - 704s 101s/step - loss: 0.6560 - acc: 0.5928 - val_loss: 0.2333 - val_acc: 0.9775
Epoch 2/5
4/6 [================>.............] - ETA: 1:17 - loss: 0.4822 - acc: 0.7165

In [ ]:
for i in range(0, len(categories)):
    train_one_category(categories[i])